## Вариант 3, Лера Бунтякова

Вариант 3: Пользователь задает стартовую страницу Википедии. Необходимо вытащить все термины, отстоящие по ссылкам от данной страницы не далее некоторого порогового расстояния.

Википедия сегодня работала против меня и не давала вынимать много ссылок, поэтому там в одном месте написано ограничение на количество ссылок на странице

In [1]:
from bs4 import BeautifulSoup
import requests
import bs4
import time

In [2]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [3]:
# мне надо было взять н случайных элементов словаря и я просто нашла эту функцию на стек оверфлоу
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [4]:
# штука с временем должна была решить проблему того, что википедия не дает мне ничего делать, но не решила
def get_page(url):

    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    r = session.get(url)
    return r.text

In [5]:
def find_links_on_page(page):
    soup = BeautifulSoup(get_page(page))
    soup = soup.find('div', {'id':'mw-content-text', 'lang':'ru', 'dir':'ltr', 'class':'mw-content-ltr'})
    if not soup:
        return
    links = soup.find_all('a')
    new_links = {}
    for link in links:
        if 'href' in link.attrs and 'title' in link.attrs:
            next_link = link.attrs['href']
            title = link.attrs['title']
            new_links[title] = next_link
    return new_links

In [6]:
def go_to_next_page(where_from, new_links, i, max_i, term_list):
    if not new_links:
        return
    keys_list = take(5, new_links.keys()) # чтобы хоть что-то работало, будем брать на каждом уровне по пять терминов
    i_0 = i

    for title in keys_list:
        term_list.append(title)

        i = i_0
        
        if i < max_i:
            link = 'https://ru.wikipedia.org' + new_links[title]
            next_links = find_links_on_page(link)
            where_from = link
            i += 1
            go_to_next_page(where_from, next_links, i, max_i, term_list)
            
        else:
            i = i_0
            continue


In [7]:
def main():

    page = input()
    max_i = 2 # максимальная глубина
    i = 0

    new_links = find_links_on_page(page)
    term_list = []
    i = 1
    where_from = page
    go_to_next_page(where_from, new_links, i, max_i, term_list)
    
    return term_list

In [8]:
main()

https://ru.wikipedia.org/wiki/%D0%A1%D1%83%D0%BC%D0%B0%D0%B9%D0%B8%D0%BB%D1%8C


['Арабский язык',
 'Алжир',
 'Бахрейн',
 'Египет',
 'Западная Сахара',
 'Иордания',
 'Ирак',
 'Арабский язык',
 'Курдские языки',
 'Флаг',
 'Герб',
 'Флаг Ирака',
 'Мухафаза (Ирак)',
 'Увеличить',
 'Ирак',
 'Мухафаза',
 'Арабский язык',
 'Провинция',
 'Дахук (мухафаза)',
 'Дахук',
 'Административное деление Ирака',
 'Ирак',
 'Курдские языки',
 'Арабский язык',
 'Высота над уровнем моря',
 'Увеличить',
 'Википедия:Ссылки на источники',
 'Редактировать раздел «История понятия»',
 'Пюиссан, Луи (страница отсутствует)',
 'en:Louis Puissant']